# Calculating my polygenic scores

In [ ]:
import traceback
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import sqlite3
from typing import List, Optional, Tuple, Dict
import pysam
import rsidx
from tqdm import tqdm
from search_your_dna.pgscatalog import read_or_download_pgs_scoring_file, PGS_METHOD_MAPPING_TO_METHOD_CATEGORIES, \
    calc_polygenic_score, calc_all_polygenic_scores

file_my_vcf = "data/GFX0237425.GRCh38.p7.annotated.hg38_multianno.updated.vcf.gz"
file_my_vcf_rsidx = "data/GFX0237425.GRCh38.p7.annotated.hg38_multianno.updated.vcf.rsidx"

In [ ]:
pgs, pgs_df = calc_polygenic_score(max_pgs_alleles=1000, pgs_file="data/pgs/PGS000034.txt.gz", file_my_vcf=file_my_vcf)
pgs

## Calc scores which have rsids in the pgs score files

In [ ]:
all_pgs_scores, errors = calc_all_polygenic_scores(files=list(glob("data/pgs/PGS00*.txt.gz")), file_my_vcf=file_my_vcf)

In [ ]:
all_pgs_scores.to_csv("data/pgs_results.csv", index=None, sep="\t")

In [ ]:
all_pgs_scores = all_pgs_scores.sort_values(by=["score","pgs_id"])


## Calc scores which have only chrom/pos values in the pgs score files

### Download rsid to hg19 chrom-pos metadata files with annovar

In [ ]:
!~/bin/annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar avsnp150 data/humandb/

### Create tabix index for metadata file

In [ ]:
!bgzip -c data/humandb/hg19_avsnp150.txt > data/humandb/hg19_avsnp150.txt.gz
!tabix --begin 2 --end 3 --sequence 1 data/humandb/hg19_avsnp150.txt.gz

### Calculate pgs score
*TODO*

In [ ]:
hg19_rsid_chrom_pos_mapping_file = "data/humandb/hg19_avsnp150.txt.gz"

hg19_vcf = pysam.Tabixfile(hg19_rsid_chrom_pos_mapping_file)

for rsid in hg19_vcf.fetch("1", 10000, 10200):
    print(rsid)